# このNotebookについて
- このNotebookは、Riiidを解く前に知っておくとよいものをまとめたものです。
- 必要と思ったものがあれば随時更新します(一部、作成中のところもありますがご了承ください)
- 筆者は、英語が得意ではないため、誤り翻訳、誤解などが発生する可能性があります。

# Riiidコンペについて

## コンペ主催者
- [RiiidLabs](https://riiidlabs.ai/solutions)
    - AIを活用して、あらゆる業界の問題解決をする企業のようです。現在は、教育問題に取り組んでいるようだ。
    - 以下の情報より、Riiidは、AIによって個人最適化されたTOEIC学習アプリを提供する韓国初スタートアップとのこと
    - https://sunryse.co/app/startups/r_6WjOnZ6mV7JPbpY
    
## 知っておくと戦いやすくなる知識(のはず)

- ここの項目に書くことは推測です。
- [riiid](https://www.riiid.co/en/product)からリンクをたどると[TOEIC学習アプリ・santatoeic](https://santatoeic.jp/)にたどり着きます。
- santatoeicは登録無料である程度、テストや講義を試すことができます。
- ためした結果、実は、今回のコンペのデータ構成と、santatoeicを比較するとかなり類似しているように思えます。
- 類似している箇所を気づいた限り挙げると以下のとおりです
    - lectures.csvのpartの数が７つと、santatoeiと一致する
    - questions.csvのpartは、TOEICテストの関連セクション
    - santatoeicにも診断テスト、講義がある。
    - santatoeicは、AIチューターシステムを採用している。
- つまり、今回のコンペはsantatoeicと深く関係があるかもしれません。
- よって、このコンペのデータ構成がイメージしづらいときは、santatoeicを使ってみるとイメージしやすくなります。
    
## 前提条件

- これ以降の説明は、学生達がTOEIC学習アプリを使って診断テスト、講義を受けている場面を想定して記載します。


## 主催者がコンペを開催する意図(目的)

- (注意)以下に書く内容は、筆者の解釈です。
- Riiid社は、学生のレベルに合わせたコンテンツ(質問、講義)を提供したい。
- それを実現するため、ユーザの使用状況(問題回答状況、正解率など)から、最適なコンテンツを提供を目指している。
- コンペのお題は、過去の正解状況などを元に、質問に答えられるかを予測します。
- 予測の目的は、AIチューターシステムが、学生のレベルに合わせたコンテンツを作るためと推測されます。



## コンペの概要

- Riiid! Answer Correctness Prediction
    - 生徒は、次の問題に答えられるか！AIの力で予測してください
- Track knowledge states of 1M+ students in the wild
    - EdNet上の100万人以上の学生の知識状態(問題回答状況、正解率など)を追跡する

## コンペが開催される背景

現在、2億6000万人の子供たちが学校に通っていません。
これらの若い学生の半数以上は、最低限の読書と数学の基準を満たしていませんでした。
COVID-19のこともあり 現在の教育システムを再構築する必要がある。

## コンペでやるべきこと

生徒が将来の相互作用でどのように行動するかを正確に予測することです。 
RiiidのEdNetデータを使用して、機械学習スキルを活用して予測します。

そして、時間の経過に伴う学生の知識のモデリングである「Knowledge Tracing(知識追跡)」のアルゴリズムを作成することです。


受賞(優勝？)チームは、2021年2月に開催されるAI教育に関するAAAI-2021ワークショップ（AIによるCOVID後の教育の想像）でモデルを発表するよう招待されます。

# コンペで使用するデータ

In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## train.csv(訓練データ)

In [2]:
#データ多いため100万行まで読み込んで見る
train_df = pd.read_csv("../input/riiid-test-answer-prediction/train.csv",nrows=10**6 )
train_df

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False
...,...,...,...,...,...,...,...,...,...,...
999995,999995,26482248,20949024,8803,0,29,1,1,14000.0,True
999996,999996,26516686,20949024,4664,0,30,3,1,17000.0,True
999997,999997,26537967,20949024,4108,0,31,1,0,18000.0,True
999998,999998,26590240,20949024,5014,0,32,3,0,6000.0,True


In [3]:
#uniquの数を出そう
train_df.nunique()

row_id                            1000000
timestamp                          769357
user_id                              3824
content_id                          13320
content_type_id                         2
task_container_id                    7740
user_answer                             5
answered_correctly                      3
prior_question_elapsed_time          1659
prior_question_had_explanation          2
dtype: int64

In [4]:
#uniqueの割合を確認
train_df['content_type_id'].value_counts().to_dict()

{0: 980093, 1: 19907}

In [5]:
#1ユーザーあたりのレコード数を出してみる
#多い人だと1万件のレコードが存在するようだ
train_df['user_id'].value_counts(sort=True,ascending=False)

7171715     10826
18122922     7593
1283420      7569
9418512      7482
4421282      6962
            ...  
1946295         3
2148001         2
15960740        2
1880240         1
19887248        1
Name: user_id, Length: 3824, dtype: int64

In [6]:
train_df['task_container_id'].value_counts(sort=True,ascending=False)

14      7845
15      7806
4       6753
5       6732
6       6722
        ... 
6328       1
6327       1
6326       1
6325       1
7739       1
Name: task_container_id, Length: 7740, dtype: int64

In [7]:
train_df['prior_question_had_explanation'].value_counts(sort=True,ascending=False).to_dict()

{True: 889887, False: 106297}

In [8]:
#content_idとtask_container_idの関係がよくわからないので調べてみる
#content_idとtask_container_idがイコール関係は100万行中168件のみ、少なくとも
#少なくともcontent_idとtask_container_idが同じ意味でないことがわかる
train_df[(train_df["content_id"] ==train_df['task_container_id'])]


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
4719,4719,6824338210,24418,2256,0,2256,2,1,24666.0,True
11350,11350,835542276,107002,215,0,215,3,1,12000.0,True
15047,15047,2319348213,141455,396,0,396,0,1,18000.0,True
17079,17079,10069776264,166728,254,0,254,3,1,28000.0,True
18377,18377,1217522351,220268,422,0,422,1,0,47000.0,True
...,...,...,...,...,...,...,...,...,...,...
978768,978768,737036336,20312279,316,0,316,1,1,18000.0,True
980078,980078,46122107583,20472541,19,0,19,0,1,25000.0,True
981605,981605,34025750562,20543516,251,0,251,1,1,18000.0,True
985079,985079,649537383,20646079,93,0,93,3,1,21000.0,True


In [9]:
#ためしにcontent_id==1のデータに絞って出力してみよう
#content_id=1は、何種類かのtask_container_idに属していることがわかる
train_df[train_df["content_id"] ==1]


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
6916,6916,10448139274,24418,1,0,3858,1,1,21000.0,True
46746,46746,9614639,1244479,1,0,20,1,1,18000.0,True
60546,60546,12181447533,1283420,1,0,4456,1,1,22000.0,True
102845,102845,470372840,2134565,1,0,138,1,1,24000.0,True
122073,122073,2276067669,2500046,1,0,246,1,1,26000.0,True
144890,144890,2262861852,3036976,1,0,412,0,0,21000.0,True
162094,162094,1247394,3392192,1,0,20,1,1,14000.0,True
166881,166881,95743304,3556553,1,0,32,1,1,24000.0,True
174406,174406,1685529091,3749425,1,0,145,1,1,16000.0,True
185261,185261,14098789587,3838215,1,0,2473,1,1,29000.0,True


In [10]:
#さらにtask_container_id==２０のデータに絞って出力してみよう
#task_container_idは、複数のcontent_idを持つことがわかった
train_df[train_df["task_container_id"] ==20].sort_values('user_id')

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
20,20,490100,115,4,0,20,1,0,29000.0,False
116,116,5533670,5382,118,0,20,0,0,26000.0,True
243,243,146310830,8623,8712,0,20,0,0,17000.0,True
396,396,3082759570,12741,6246,0,20,3,0,21000.0,True
644,644,1195437,13134,10687,0,20,0,0,19000.0,True
...,...,...,...,...,...,...,...,...,...,...
998949,998949,46305702,20913319,7979,0,20,1,0,23000.0,True
999365,999365,658478,20913864,9083,0,20,0,0,27000.0,True
999400,999400,1457395,20938253,4639,0,20,0,1,10000.0,True
999921,999921,2516547681,20948951,11350,0,20,2,1,34000.0,True


In [11]:
#一人のユーザは、どのようなデータを持つのか確認しよう
#まず、レコード数(=質問の数が多い)が多いユーザを特定する
#とりあえずuser_id=1283420あたりをサンプルとする(id=7171715は、極端に値が大きい。異常値？)
user_df = train_df[train_df.answered_correctly != -1].groupby('user_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
user_df.columns = ['user_id', 'user_questions', 'user_mean']
user_df.sort_values('user_questions')

,user_id,user_questions,user_mean
3661,19887248,1,0.000000
308,1880240,1,1.000000
2932,15960740,2,0.500000
366,2148001,2,0.500000
318,1946295,3,0.666667
...,...,...,...
796,4421282,6960,0.757328
1707,9418512,7261,0.608869
3329,18122922,7413,0.708080
218,1283420,7476,0.828785


In [12]:
#説明にある通り、１つのtask_container_idは、３つのcontent_idを持つことがわかった
train_df[train_df["user_id"] ==1283420]

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
54058,54058,0,1283420,3822,0,0,1,1,NaN,NaN
54059,54059,29795,1283420,638,0,1,0,1,11000.0,False
54060,54060,91982,1283420,4231,0,2,0,0,28000.0,False
54061,54061,117342,1283420,7978,0,3,3,0,60000.0,False
54062,54062,145733,1283420,8225,0,4,1,1,23000.0,False
...,...,...,...,...,...,...,...,...,...,...
61622,61622,22364257029,1283420,2041,0,5260,0,1,16000.0,True
61623,61623,22364257029,1283420,2039,0,5260,3,1,16000.0,True
61624,61624,22373215518,1283420,3401,0,5261,1,1,26333.0,True
61625,61625,22373215518,1283420,3400,0,5261,2,0,26333.0,True


In [14]:
#CSV作成用に読み込み
train_df_csv = pd.read_csv("../input/riiid-test-answer-prediction/train.csv",nrows=1000)
train_df_csv.to_csv("train_df_1000.csv")

In [25]:
train_df['user_answer'].value_counts()

 0    278205
 1    267017
 3    257932
 2    176939
-1     19907
Name: user_answer, dtype: int64

## カラムの説明
### row_id(int64)
>ID code for the row.

行に対するID

### timestamp(int64)
> the time in milliseconds between this user interaction and the first event completion from that user.

このユーザーインタラクション(ユーザの新規登録)から、該当レコードのイベント完了までのミリ秒単位の時間。
訓練データの１つのuser_idに着目すると、timestampがレコード発生ごとに増えていくのがわかるかと思います。


### user_id: (int32) 
> ID code for the user.

ユーザーのIDコード。

### content_id: (int16) 
> ID code for the user interaction

ユーザーインタラクションのIDコード。
「インタラクション」だと分かりづらい表現だから、単にコンテンツIDで良いと思う。



### content_type_id: (int8) 
> 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

イベントがユーザーに提示された質問の場合は0、イベントが講義を見ているユーザーの場合は1。
100万件のデータで比較すると、件数は以下のようになっている。
{0: 980093, 1: 19907}

### task_container_id: (int16) 
> Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

質問または講義のバッチのIDコード。
たとえば、ユーザーは、いずれかの説明を表示する前に、3つの質問を連続して表示する場合があります。
これら3つはすべてtask_container_idを共有します。

### user_answer: (int8) 
> the user's answer to the question, if any. Read -1 as null, for lectures.

質問に対するユーザーの回答(数字で答えるようだ)。講義の場合は、-1を「null」として読み取ります。
※質問した回数ではない(筆者は最初、質問の回数のことだと思っていた。)

### answered_correctly: (int8) 
> if the user responded correctly. Read -1 as null, for lectures.

ユーザーが正しく応答した場合。講義の場合は、-1をnullとして読み取ります。

### prior_question_elapsed_time: (float32) 
> The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

ユーザーが前の質問バンドルの各質問に回答するのにかかった平均時間（ミリ秒単位）。
その間の講義は無視されます。ユーザーの最初の質問バンドルまたは講義ではnullです。
時間は、ユーザーが前のバンドルの各質問を解決するのにかかった平均時間であることに注意してください。

### prior_question_had_explanation: (bool) 
> Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.


ユーザーが前の質問バンドルに回答した後、説明と正しい回答を見たかどうか。その間の講義は無視されます。
値は単一の質問バンドル間で共有され、ユーザーの最初の質問バンドルまたは講義ではnullになります。通常、ユーザーに表示される最初のいくつかの質問は、フィードバックが得られなかったオンボーディング診断テストの一部でした。

要するに、前回、質問に回答した後に解説をみたかどうか。最初の質問のときは、(当然のことながら)解説を見れないからnullになる。
実際の値は以下の通り。<br>
{True: 889887, False: 106297}


## questions.csv(質問データ)

In [15]:
questions_df = pd.read_csv("../input/riiid-test-answer-prediction/questions.csv")
questions_df

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


In [26]:
#uniquの数を出そう
questions_df.nunique()


question_id       13523
bundle_id          9765
correct_answer        4
part                  7
tags               1519
dtype: int64

In [17]:
questions_df['bundle_id'].value_counts(sort=True,ascending=False)

8034     5
7790     5
7260     5
7195     5
7190     5
        ..
5058     1
5057     1
5056     1
5055     1
13522    1
Name: bundle_id, Length: 9765, dtype: int64

In [18]:
#questions.csvを見ていると、question_idとbundle_idは、同じ値になっているように見える
#本当かどうか確認する
#確認した結果、同一bundle_idに、いくつかのquestion_idが紐づくことがわかる。
questions_df[questions_df["question_id"] != questions_df["bundle_id"]]

,question_id,bundle_id,correct_answer,part,tags
1401,1401,1400,0,3,136 92 102
1402,1402,1400,1,3,82 92 102
1404,1404,1403,0,3,136 38 29
1405,1405,1403,3,3,82 38 29
1407,1407,1406,3,3,136 38 102
...,...,...,...,...,...
13246,13246,13244,1,3,136 81 92
13248,13248,13247,2,3,136 81 92
13249,13249,13247,1,3,136 81 92
13251,13251,13250,2,3,136 81 92


In [19]:
#ためしに bundle_id=7795のデータを見てみよう
#大問題=bundle_id, 小問題=question_idと考えて良さそうだ
#partが全て一致、tagsもほぼ一致していることがわかる
questions_df[questions_df["bundle_id"] == 7795]

,question_id,bundle_id,correct_answer,part,tags
7795,7795,7795,1,7,145 160 16 22 122
7796,7796,7795,2,7,118 160 16 22 122
7797,7797,7795,1,7,97 160 16 22 122
7798,7798,7795,2,7,97 160 16 22 122
7799,7799,7795,1,7,97 160 16 22 122


In [34]:
#t = questions_df[questions_df['question_id'] == 7795].tags
questions_df.tags.value_counts()

8                 738
73                617
53                523
1                 413
96                373
                 ... 
157 144 81          1
157 144 38          1
106 169 162 81      1
157 169 162 81      1
157 169 92          1
Name: tags, Length: 1519, dtype: int64

### question_id: 
> foreign key for the train/test content_id column, when the content type is question (0).

コンテンツタイプが質問ならば0、question_idは、train / testの「content_id」列の外部キー。


### bundle_id:
> code for which questions are served together.

質問が一緒に提供されるコード。
大問題=bundle_id, 小問題=question_idというイメージ。

### correct_answer:
> the answer to the question. Can be compared with the train user_answer column to check if the user was right.

質問への回答。 train.csvの「user_answer」列と比較して、ユーザーが正しいかどうかを確認できます

### part: 
> the relevant section of the TOEIC test.

TOEICテストの関連セクション。

### tags: 
> one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

質問の1つ以上の詳細なタグコード。タグの意味は提供されませんが、これらのコードは質問をまとめるのに十分です。


# lectures.csv(講義のメタデータ)


ユーザーが教育を進めるときに視聴する講義のメタデータ。

In [20]:
lectures_df = pd.read_csv("../input/riiid-test-answer-prediction/lectures.csv")
lectures_df

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question
...,...,...,...,...
413,32535,8,5,solving question
414,32570,113,3,solving question
415,32604,24,6,concept
416,32625,142,2,concept


In [27]:
lectures_df['type_of'].value_counts()

concept             222
solving question    186
intention             7
starter               3
Name: type_of, dtype: int64

In [28]:
lectures_df['tag'].value_counts()

136    7
27     6
161    6
74     6
134    6
      ..
90     1
112    1
31     1
138    1
2      1
Name: tag, Length: 151, dtype: int64

### lecture_id: 
> foreign key for the train/test content_id column, when the content type is lecture (1).

コンテンツタイプが講義（1）の場合、train / testcontent_id列の外部キー。

### part:
>  top level category code for the lecture.

講義のトップレベルのカテゴリコード。

### tag:
>  one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

講義用の1つのタグコード。タグの意味は提供されませんが、これらのコードは講義をまとめるのに十分です。

### type_of:
>  brief description of the core purpose of the lecture

講義の主な目的の簡単な説明


## example_test_rows.csv(テストセットデータ)

時系列APIによって配信されるテストセットデータの3つのサンプルグループ。形式はtrain.csvとほぼ同じです。 AIチューターがいつでも実際に利用できる情報を反映する、2つの異なる列がありますが、一度に1人のユーザーの情報を厳密に表示するのではなく、APIパフォーマンスのためにユーザーインタラクションをグループ化します。一部のユーザーは、訓練データに表示されていない非表示のテストセットに表示され、Webサイトへの新着のモデリングにすばやく適応するという課題をエミュレートします。


In [21]:
test_df = pd.read_csv("../input/riiid-test-answer-prediction/example_test.csv")
test_df

,row_id,group_num,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,0,0,0,275030867,5729,0,0,NaN,NaN,[],[]
1,1,0,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN
2,2,0,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN
3,3,0,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN
4,4,0,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
99,104,3,13167339284,1900527744,3004,0,1179,24667.0,True,NaN,NaN
100,105,3,13167339284,1900527744,3003,0,1179,24667.0,True,NaN,NaN
101,106,3,64497673060,7792299,7908,0,676,19000.0,True,NaN,NaN
102,107,3,62798166743,288641214,9077,0,269,25000.0,True,NaN,NaN


### prior_group_responses (string)
>provides all of the user_answer entries for previous group in a string representation of a list in the first row of the group. All other rows in each group are null. If you are using Python, you will likely want to call eval on the non-null rows. Some rows may be null, or empty lists.

prior_group_responses（string）は、前のグループのすべてのuser_answerエントリを、グループの最初の行のリストの文字列表現で提供します。各グループの他のすべての行はnullです。 Pythonを使用している場合は、null以外の行でevalを呼び出すことをお勧めします。一部の行はnullまたは空のリストである可能性があります。


### prior_group_answers_correct (string)
> provides all the answered_correctly field for previous group, with the same format and caveats as prior_group_responses. Some rows may be null, or empty lists.

prior_group_answers_correct（string）は、prior_group_responsesと同じ形式と警告で、前のグループのすべてのanswered_correctlyフィールドを提供します。一部の行はnullまたは空のリストである可能性があります。

## example_sample_submission.csv(提出サンプルファイル)

In [22]:
submission_df = pd.read_csv("../input/riiid-test-answer-prediction/example_sample_submission.csv")
submission_df

,row_id,answered_correctly,group_num
0,0,0.5,0
1,1,0.5,0
2,2,0.5,0
3,3,0.5,0
4,4,0.5,0
...,...,...,...
99,104,0.5,3
100,105,0.5,3
101,106,0.5,3
102,107,0.5,3


# 時系列APIの詳細(Time-series API Details)

> Refer to the starter notebook for an example of how to complete a submission. The time-series API has changed somewhat from previous competitions!

提出を完了する方法の例については、[スターターノートブック](https://www.kaggle.com/sohier/competition-api-detailed-introduction)を参照してください。時系列APIは、以前のコンテストから多少変更されました。


> You should not try to submit anything for the rows that contain lectures.

講義を含む行には何も送信しようとしないでください。


> The API provides user interactions groups in the order in which they occurred. Each group will contain interactions from many different users, but no more than one task_container_id of questions from any single user. Each group has between 1 and 1000 users.

APIは、ユーザーインタラクショングループを発生順に提供します。各グループには、多くの異なるユーザーからのインタラクションが含まれますが、単一のユーザーからの質問のtask_container_idは1つだけです。各グループには1〜1000人のユーザーがいます。


> Expect to see roughly 2.5 million questions in the hidden test set.

非表示のテストセットで約250万の質問が表示されると予想されます。


> The API will load up to 1 GB of the test set data in memory after initialization. The initialization step (env.iter_test()) will require meaningfully more memory than that; we recommend you do not load your model until after making that call. The API will also consume roughly 15 minutes of runtime for loading and serving the data, but will also obfuscate the true runtime for all submissions.

APIは、初期化後に最大1GBのテストセットデータをメモリにロードします。初期化ステップ（env.iter_test（））は、それよりも意味のある量のメモリを必要とします。その呼び出しを行うまで、モデルをロードしないことをお勧めします。 APIは、データのロードと提供に約15分のランタイムも消費しますが、すべての送信の実際のランタイムを難読化します。

> The API loads the data using the types specified above (int32 for user_id, int8 for content_type_id, etc).

APIは、上記で指定されたタイプ（user_idの場合はint32、content_type_idの場合はint8など）を使用してデータをロードします。

